In [1]:
# %matplotlib notebook
from IPython.display import display, IFrame, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy             as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from matplotlib          import rc 
from matplotlib          import cm

import pandas as pd
import corner

import os
import os.path as path
import wget

import emcee

from tqdm import tqdm

from scipy.integrate import dblquad
from scipy.integrate import quad

In [2]:
# Descargamos data
if not path.exists('Data'):
    os.mkdir('Data')

!rm Data/*

file = 'Data/Gaussiana.csv'
url = 'https://raw.githubusercontent.com/asegura4488/Database/main/MetodosComputacionalesReforma/Gaussiano.csv'
if not path.exists(file):
    Path_ = wget.download(url,file)
    print('File loaded')
else:
    Path_ = file

"rm" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [3]:
data = pd.read_csv(Path_)
data

,x
0,0.906451
1,5.446117
2,6.840238
3,0.743644
4,7.338518
...,...
95,1.497647
96,4.236658
97,4.678540
98,6.479197


In [4]:
X = data.x

In [5]:
def LogPrior(mu, sigma):
    
    if 3. <= mu <= 5. and 0.5 <= sigma <= 3.5:
        return 0.
    else:
        return -np.inf

In [6]:
def Gauss(mu, sigma, x):
    
    return np.exp( -0.5*(x-mu)**2/sigma**2  )/np.sqrt(2*np.pi*sigma**2)

In [7]:
def Likelihood(mu, sigma, x):
    
    return Gauss(mu,sigma,x)

In [8]:
def JointLikelihood(mu, sigma, x):
    return np.sum( np.log(Likelihood(mu, sigma, x)) )

In [9]:
def LogPosterior(mu, sigma, x):
    
    LogP = LogPrior(mu, sigma)
    
    if not np.isfinite(LogP):
        return -np.inf
    else:
        return JointLikelihood(mu, sigma, x) + LogP

In [10]:
def Metropolis(mu, sigma, X, Posterior, NSteps=int(2e4 + 1), delta= 0.4):
    
    x = np.zeros((NSteps,2))
    
    # Prior
    x[0, 0] = mu
    x[0, 1] = sigma
    
    for i in tqdm(range(1,NSteps)):
        
        P0 = Posterior(x[i-1, 0], x[i-1, 1], X)
        
        muf = x[i-1, 0] + delta*2*(np.random.rand()-0.5)
        sigmaf = x[i-1, 1] + delta*2*(np.random.rand()-0.5)
        
        P1 = Posterior(muf, sigmaf, X)
        
        if P1 == -np.inf:
            alpha = P1
        else:
            alpha = np.minimum( 1, P1/P0 )
        
        g = np.random.rand()
        
        if alpha > g:
            x[i, 0] = muf
            x[i, 1] = sigmaf
        else:
            x[i,:] = x[i-1,:]
            
    return x[1000:,:]

In [11]:
muestreo = Metropolis(4.5, 1.5, X, LogPosterior)

100%|██████████| 20000/20000 [00:23<00:00, 851.68it/s]


In [12]:
mediana_mu = np.percentile(muestreo[:, 0], 50)
mediana_mu

4.0048193843615545

In [13]:
liminf_mu = np.percentile(muestreo[:, 0], 16)
liminf_mu

3.284482150257676

In [14]:
limsup_mu = np.percentile(muestreo[:, 0], 84)
limsup_mu

4.693522647396864

In [15]:
mediana_sigma= np.percentile(muestreo[:, 1], 50)
mediana_sigma

1.7988304456414506

In [16]:
liminf_sigma = np.percentile(muestreo[:, 1], 16)
liminf_sigma

0.7173030348234692

In [17]:
limsup_sigma = np.percentile(muestreo[:, 1], 84)
limsup_sigma

3.0015386026673605